In [21]:
class genericField:
  def __init__(self, bbox=(0, 0, 0, 0),fieldType=None):
    self.fieldType=fieldType
    self.BoundingBox = {
            "Left": bbox[0],  
            "Top": bbox[1], 
            "Width": bbox[2],
            "Height": bbox[3]
            }
  def __str__(self):
    return str(self._dict_)
  def __repr__(self):
    return self._str_()

class dateField(genericField):
  def __init__(self, bbox=(0, 0, 0, 0)):
    super(dateField, self).__init__(bbox,"date")
    self.date={
        "subFields" : 0,       #3 for dd/mm/yyyy
        "side" : 0,
        "count" : []          #[2,2,4] for dd/mm/yyyy
    }

class blocksField(genericField):
  def __init__(self,bbox=(0, 0, 0, 0)):
    super(blocksField, self).__init__(bbox,"blocksField")
    self.blocks={
        "side":0, # side length of each block
        "count":0 # number of blocks
    }

In [22]:
import cv2
import numpy as np
from PIL import Image
        
img1 = cv2.imread('formexample6.png')
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img,(1,3),0)
img_bin,ab = cv2.threshold(img,220,255,cv2.THRESH_BINARY )
cv2.imwrite("Thresholded_img.jpg",ab)
binary_map = 255-ab

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_map, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 190:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result
cv2.imwrite("Thresholded_img.jpg",result)
img_bin,result = cv2.threshold(result,244,255,cv2.THRESH_BINARY )
kernel = np.ones((3, 3), np.uint8)   
result = cv2.erode(result, kernel,iterations = 5) 
result = cv2.GaussianBlur(result,(3,3),0)
cv2.imwrite("Image_bin_new.jpg",result)
boxes = list()
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if h<30 and abs(w-h)<6 and w>18.5 and h>18 and w<30:
          new_img = img[y:y+h, x:x+w]
          cv2.rectangle(img1, (x, y), (x+w, y+h), (255, 0, 0), 1)
          ob = blocksField((x,y,w,h))
          boxes.append(ob)
cv2.imwrite("RESULT1.jpg",img1)
cv2.imshow("IMAGE",img1)
cv2.waitKey(0)

-1